# Exp 2 – Support Vector Regessor 

##

### Nhập các thư viện cho mô hình

In [10]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

# nhóm công cụ chia giữ liệu và đánh giá mô hình
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

# đánh giá mô hình
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.metrics import r2_score


### Tải lên file dữ liệu đã qua EDA và Preprocessing

In [11]:
train_path = "../../data/data_afterProcessed/train_processed.csv"
test_path = "../../data/data_afterProcessed/test_processed.csv"
target_path = "../../data/data_afterProcessed/target.csv"

train_processed = pd.read_csv(train_path)
test_processed = pd.read_csv(test_path)
target = pd.read_csv(target_path)

print("Đã tải các tệp dữ liệu đã qua xử lý")

Đã tải các tệp dữ liệu đã qua xử lý


### Logarit hoá biến mục tiêu

In [12]:
y_log = np.log1p(target)
print("Đã log-transform SalePrice")

Đã log-transform SalePrice


##### thay đổi này giúp mô hình học nhanh hơn

### Chia dữ liệu và huấn luyện mô hình

In [13]:
from sklearn.model_selection import train_test_split

features_train, features_test, target_train, target_test = train_test_split(
    train_processed, target, test_size=0.2, random_state=25
)

print("Đã chia dữ liệu tệp train và test để huấn luyện và đánh giá mô hình")

Đã chia dữ liệu tệp train và test để huấn luyện và đánh giá mô hình


In [14]:
model_SVR = SVR(
    kernel='rbf',
    C=1.0,
    epsilon=0.1,
    gamma='scale',
    verbose=True
)

model_SVR.fit(features_train, target_train.values.ravel())

print("Đã huấn luyện xong mô hình Support Vector Regression")

[LibSVM]Đã huấn luyện xong mô hình Support Vector Regression


### Đánh giá mô hình

In [15]:
# Dự đoán trên tập train
y_pred_train = model_SVR.predict(train_processed)

# R² score trên giá trị gốc
r2_original = r2_score(target, y_pred_train)

# Chuyển sang log scale để đánh giá
y_pred_log = np.log1p(y_pred_train)
rmse_log = np.sqrt(mean_squared_error(y_log, y_pred_log))
r2_log = r2_score(y_log, y_pred_log)

print(f"RMSE (log scale): {rmse_log:.4f}")
print(f"R² (log scale): {r2_log:.4f}")
print(f"R² trên tập train: {r2_original:.4f}")

RMSE (log scale): 0.0075
R² (log scale): 0.9385
R² trên tập train: 0.9407


### Lưu lại file đánh giá và file mô hình

In [16]:
test_pred_log = model_SVR.predict(test_processed)

if test_pred_log.ndim > 1:
    test_pred_log = test_pred_log.ravel()

test_pred = np.expm1(test_pred_log)

submission = pd.DataFrame({
    'Id': range(1461, 1461 + len(test_pred)),  
    'SalePrice': test_pred
})

submission.to_csv('result/SVR_submission.csv', index=False)
print("Đã tạo file SVR_submission.csv")

Đã tạo file SVR_submission.csv


In [17]:
joblib.dump(model_SVR, 'pkl/SVR_model.pkl')
print("Đã lưu mô hình SVR_model.pkl")

Đã lưu mô hình SVR_model.pkl


In [18]:
print("Phân tích thêm về mô hình SVR:")
print(f"- Kernel được sử dụng: {model_SVR.kernel}")
print(f"- Tham số C (regularization): {model_SVR.C}")
print(f"- Tham số epsilon: {model_SVR.epsilon}")
print(f"- Số lượng support vectors: {model_SVR.support_vectors_.shape[0]}")

Phân tích thêm về mô hình SVR:
- Kernel được sử dụng: rbf
- Tham số C (regularization): 1.0
- Tham số epsilon: 0.1
- Số lượng support vectors: 319
